In [2]:
import os, csv, json, requests
import glob
import pycountry
import pandas as pd
import numpy as np
from datetime import date
import pickle

In [3]:
with open(r"pv_detailsSSP2-RCP19-2050.pickle", "rb") as input_file:
        pv_pickle = pickle.load(input_file)

url_api = "https://re.jrc.ec.europa.eu/api/v5_2/PVcalc?"

coord = pd.read_csv("Coordinate_unique.csv")

In [4]:
# Nominal power of the PV system, in kW. 1kWp / m²
# peakpower

# PVgis technology. Choices are: "crystSi", "CIS", "CdTe" and "Unknown"
# pvtechchoice
# Ecoinvent technology : "single-Si", "multi-Si", "a-Si", "ribbon-Si", "CIS", "CdTe"

# Type of mounting of the PV modules are: "free" for free-standing and "building" for building-integrated.
# mountingplace
# Ecoinvent mounting : "facade"-"flat-roof"-"slanted-roof" = building, "open ground" = free

# Inclination angle from horizontal plane of the (fixed) PV system.
# angle
# Ecoinvent inclination : "facade" = 90°, "-roof" = 0°, "open ground" = no value

# Calculate the optimum inclination AND orientation angles. Value of 1 for "yes". All other values (or no value) mean "no".
# optimalinclination
# Ecoinvent : "open ground" = 1, no value for others

In [31]:
iRow = 0

peakpower = pv_pickle.iloc[iRow]["Peak power (kWp)"]

Tech_pickle = pv_pickle.iloc[iRow]["Technology"]
pvtechchoice = ("crystSi" if "Si" in Tech_pickle
                    else "CIS" if peakpower == "CIS"
                    else "CdTe" if peakpower == "CdTe"
                    else "Unknown")

Mounting_pickle = pv_pickle.iloc[iRow]["Installation"]
mountingplace = ("building" if "roof" or "facade" in Mounting_pickle
                    else "free")

angle = ("0" if "roof" in Mounting_pickle
                    else "90" if "facade" in Mounting_pickle
                    else "")

optimalinclination = ("1" if "open ground" in Mounting_pickle
                    else "")
p1 = "&peakpower="
p2 = "&pvtechchoice="
p3 = "&mountingplace="
p4 = "&angle="
p5 = "&optimalinclination="
mp = "&loss=14&outputformat=json"

In [88]:
market_region = premise_ds.iloc[iRow]["Market group location"]

coordonnees_region = coord.loc[coord['Short name'] == market_region, ['Latitude', 'Longitude']]
#coordonnees_region
E_y_values_list = []

for index, row in coordonnees_region.iterrows():
    # Extraire les valeurs de latitude et de longitude de la ligne actuelle
    lat = "lat="+"{:.2f}".format(row['Latitude'])+"&"
    lon = "lon="+"{:.2f}".format(row['Longitude'])
    url_test = base_url_coord+lat+lon+p1+str(peakpower)+p2+pvtechchoice+p3+mountingplace+p4+angle+p5+optimalinclination+mp
    # print(url_test)
    response_test = requests.get(url_test)
    # print(response_test)
    jsondata_test = response_test.json()
    if response_test.status_code == 200:
        # print("Success")
        jsondata_test = response_test.json()
        # Stocker la valeur E_y_value dans la liste
        E_y_values_list.append(jsondata_test["outputs"]["totals"]["fixed"]["E_y"])

# Créer un DataFrame à partir de la liste des valeurs E_y_value
E_y_df = pd.DataFrame({'E_y_value': E_y_values_list})

# Calculer la moyenne des valeurs E_y_value
moyenne_E_y = E_y_df['E_y_value'].mean()

# Afficher la moyenne
print("La moyenne des valeurs E_y_value est :", moyenne_E_y)

La moyenne des valeurs E_y_value est : 2321.025357142857


In [5]:
market_region = premise_ds.iloc[0]["Market group location"]
market_region

nan

In [46]:
pv_pickle = pv_pickle.head(5)
iRow = 0
while iRow <5:
    for index, row in df.iterrows():
        if str(pv_pickle.iloc[iRow]["Market group location"]) == "nan":
                print("NaN")
                iRow = iRow+1
        else:
            peakpower = pv_pickle.iloc[iRow]["Peak power (kWp)"]
            
            Tech_pickle = pv_pickle.iloc[iRow]["Technology"]
            pvtechchoice = ("crystSi" if "Si" in Tech_pickle
                                else "CIS" if peakpower == "CIS"
                                else "CdTe" if peakpower == "CdTe"
                                else "Unknown")
            
            Mounting_pickle = pv_pickle.iloc[iRow]["Installation"]
            mountingplace = ("building" if "roof" or "facade" in Mounting_pickle
                                else "free")
            
            angle = ("0" if "roof" in Mounting_pickle
                                else "90" if "facade" in Mounting_pickle
                                else "")
            
            optimalinclination = ("1" if "open ground" in Mounting_pickle
                                else "")
            p1 = "&peakpower="
            p2 = "&pvtechchoice="
            p3 = "&mountingplace="
            p4 = "&angle="
            p5 = "&optimalinclination="
            mp = "&loss=14&outputformat=json"
    
            market_region = pv_pickle.iloc[iRow]["Market group location"]
            
            coordonnees_region = coord.loc[coord['Short name'] == market_region, ['Latitude', 'Longitude']]
            #coordonnees_region
            E_y_values_list = []
            
            for index, row in coordonnees_region.iterrows():
                # Extraire les valeurs de latitude et de longitude de la ligne actuelle
                lat = "lat="+"{:.2f}".format(row['Latitude'])+"&"
                lon = "lon="+"{:.2f}".format(row['Longitude'])
                url_test = base_url_coord+lat+lon+p1+str(peakpower)+p2+pvtechchoice+p3+mountingplace+p4+angle+p5+optimalinclination+mp
                # print(url_test)
                response_test = requests.get(url_test)
                # print(response_test)
                jsondata_test = response_test.json()
                if response_test.status_code == 200:
                    # print("Success")
                    jsondata_test = response_test.json()
                    # Stocker la valeur E_y_value dans la liste
                    E_y_values_list.append(jsondata_test["outputs"]["totals"]["fixed"]["E_y"])
            
            moyenne_E_y = np.mean(E_y_values_list)
            
            # Afficher la moyenne
            print("La moyenne des valeurs E_y_value est :", moyenne_E_y)
            iRow = iRow+1

NaN
NaN
La moyenne des valeurs E_y_value est : 5235.092333333333
La moyenne des valeurs E_y_value est : 5235.092333333333
La moyenne des valeurs E_y_value est : 4906.191


IndexError: single positional indexer is out-of-bounds

In [54]:
# Ecoinvent technology : "single-Si", "multi-Si", "a-Si", "ribbon-Si", "CIS", "CdTe"
# Ecoinvent mounting : "facade"-"flat-roof"-"slanted-roof" = building, "open ground" = free

tech_list = [' single-Si', ' multi-Si', ' a-Si', ' ribbon-Si', ' CIS', ' CdTe']
mounting_list = ['facade', 'flat-roof', 'slanted-roof', 'open ground']

# Initialiser une troisième liste pour stocker les suites
tech_mount_list = []

# Boucle pour générer chaque suite et les stocker dans la troisième liste
for element1 in tech_list:
    for element2 in mounting_list:
        suite = element1 +" "+ element2
        tech_mount_list.append(suite)

# Afficher les suites stockées dans la troisième liste
print(tech_mount_list)


[' single-Si facade', ' single-Si flat-roof', ' single-Si slanted-roof', ' single-Si open ground', ' multi-Si facade', ' multi-Si flat-roof', ' multi-Si slanted-roof', ' multi-Si open ground', ' a-Si facade', ' a-Si flat-roof', ' a-Si slanted-roof', ' a-Si open ground', ' ribbon-Si facade', ' ribbon-Si flat-roof', ' ribbon-Si slanted-roof', ' ribbon-Si open ground', ' CIS facade', ' CIS flat-roof', ' CIS slanted-roof', ' CIS open ground', ' CdTe facade', ' CdTe flat-roof', ' CdTe slanted-roof', ' CdTe open ground']


In [55]:
Tech_pickle = pv_pickle.iloc[0]["Technology"]
Mounting_pickle = pv_pickle.iloc[0]["Installation"]

parameter_combination = Tech_pickle + Mounting_pickle
parameter_combination

' single-Si slanted-roof '

In [89]:
# Ecoinvent technology : "single-Si", "multi-Si", "a-Si", "ribbon-Si", "CIS", "CdTe"
# Ecoinvent mounting : "facade"-"flat-roof"-"slanted-roof" = building, "open ground" = free

mktrgn_list = ["RUS","CHN","RSAF","INDO","JAP","RSAM","WAF","UKR","INDIA","ME","WEU","NAF","KOR","EAF","STAN","CEU","RCAM","CAN","RSAS","SAF","BRA","TUR","OCE","USA","SEAS"]

ei_tech_list = [' single-Si', ' multi-Si', ' a-Si', ' ribbon-Si', ' CIS', ' CdTe']
ei_mounting_list = ['facade', 'flat-roof', 'slanted-roof', 'open ground']

tech_list = ['crystSi', 'CIS', 'CdTe', 'Unknown']
mounting_list = ['free', 'building']
angle_list = ['90','0']
optincl = '1'
poss_list = []
test_list = []
url_list = []
yrlygen_list = []
result_list = []

p0 = "&peakpower=1"
p1 = "&pvtechchoice="
p2 = "&mountingplace="
p3 = "&angle="
p4 = "&optimalinclination="
mp = "&loss=14&outputformat=json"

# Boucle pour générer chaque suite et les stocker dans la troisième liste
for e1 in tech_list:
    for e2 in mounting_list:
        if e2 == "free":
            suite_url = p0+p1+e1+p2+e2+p4+optincl+mp
            poss_list.append((suite_url, e1+" "+e2))
        else:
            for e3 in angle_list:
                suite_url = p0+p1+e1+p2+e2+p3+e3+mp
                poss_list.append((suite_url,e1+" "+e2+" "+e3))
#print(poss_list)
for region in mktrgn_list:
    coordonnees_region = coord.loc[coord['Short name'] == region, ['Latitude', 'Longitude']]
    for x in poss_list:
        for index, row in coordonnees_region.iterrows():
            lat = "lat="+"{:.2f}".format(row['Latitude'])+"&"
            lon = "lon="+"{:.2f}".format(row['Longitude'])
            url_request = url_api + lat + lon + x[0]
            #url_list.append(url_request) 
            response_test = requests.get(url_request)
            jsondata_test = response_test.json()
            if response_test.status_code == 200:
                #print("Success")
                jsondata_test = response_test.json()
                yrlygen_list.append(jsondata_test["outputs"]["totals"]["fixed"]["E_y"])
        mean_E_y = np.mean(yrlygen_list)
        yrlygen_list = []
        result_list.append((region, x[1], mean_E_y))
        print(region,x[1],mean_E_y)
        print(result_list)
                

RUS crystSi free 1046.6707407407407
[('RUS', 'crystSi free', 1046.6707407407407)]


KeyboardInterrupt: 

In [ ]:
# PVgis technology. Choices are: "crystSi", "CIS", "CdTe" and "Unknown"
# pvtechchoice
# Ecoinvent technology : "single-Si", "multi-Si", "a-Si", "ribbon-Si", "CIS", "CdTe"

# Type of mounting of the PV modules are: "free" for free-standing and "building" for building-integrated.
# mountingplace
# Ecoinvent mounting : "facade"-"flat-roof"-"slanted-roof" = building, "open ground" = free

# Inclination angle from horizontal plane of the (fixed) PV system.
# angle
# Ecoinvent inclination : "facade" = 90°, "-roof" = 0°, "open ground" = no value

# Calculate the optimum inclination AND orientation angles. Value of 1 for "yes". All other values (or no value) mean "no".
# optimalinclination
# Ecoinvent : "open ground" = 1, no value for others